# Model Output Notebook

<img style="float:center;" src="https://arcticexpansion.vse.gmu.edu/sites/arcticexpansion.vsnet.gmu.edu/files/images/header5d2.png" width=600px>

### ADCIRC-SWAN Output


### Initialize Libraries

In [1]:
import netCDF4 as nc4;
import pathlib as pl;     
import numpy as np   

source $HOME/miniforge3/bin/activate

salloc --ntasks=5 --nodes=1 --partition=normal --time=10:00:00

### Defined Functions

#### Data for this exercise can be found here
https://doi.org/10.17603/ds2-h0fw-2p96

Download the swan_HS.63.nc from one of the 4 folders

---

In [2]:
INPUT_ROOT = pl.Path('/groups/ORC-CLIMATE/fhrl_repo/Arctic_Database/Raw_DATA')
OUTPUT_ROOT = pl.Path('/scratch/tmiesse/project/data4spatial')


In [3]:
YEARS       = [1981]
KEEP_VARS = ["time", "x", "y", "element", "zeta", "depth"]
for year in YEARS:
    src_path = INPUT_ROOT / str(year) / 'outputs' / 'fort.63.nc'
    if not src_path.exists():
        print(f"Skipping {year}: {src_path} not found")
        continue

    # Prepare output directory for this year
    out_dir = OUTPUT_ROOT / str(year)
    out_dir.mkdir(parents=True, exist_ok=True)
    dst_path = out_dir / 'fort.63.cf.nc'

    print(f"Processing year {year} → {dst_path}")

    with nc4.Dataset(src_path, 'r') as ds0, nc4.Dataset(dst_path, 'w') as ds1:
        # 1) Copy global attributes
        ds1.setncatts({k: ds0.getncattr(k) for k in ds0.ncattrs()})

        # 2) Copy dimensions
        for name, dim in ds0.dimensions.items():
            ds1.createDimension(name, len(dim) if not dim.isunlimited() else None)

        # 3) Copy selected variables, handling _FillValue
        for name in KEEP_VARS:
            var0 = ds0.variables[name]
            out_name = "face_node_connectivity" if name == "element" else name

            # Determine fill_value if present
            fv = getattr(var0, "_FillValue", None)

            # Create variable with fill_value if needed
            if fv is not None:
                var1 = ds1.createVariable(out_name,
                                          var0.datatype,
                                          var0.dimensions,
                                          fill_value=fv)
            else:
                var1 = ds1.createVariable(out_name,
                                          var0.datatype,
                                          var0.dimensions)

            # Copy the data
            var1[:] = var0[:]

            # Copy all attributes except _FillValue
            for attr, val in var0.__dict__.items():
                if attr == "_FillValue":
                    continue
                var1.setncattr(attr, val)

            # Mark connectivity variable
            if name == "element":
                var1.setncattr("cf_role", "face_node_connectivity")

        # 4) Declare the mesh topology
        mesh = ds1.createVariable("mesh2d", "i4", ())
        mesh.setncattr("cf_role", "mesh_topology")
        mesh.setncattr("topology_dimension", 2)
        mesh.setncattr("node_coordinates", "x y")
        mesh.setncattr("face_node_connectivity", "face_node_connectivity")

    print(f"  → Done {year}")

print("All years processed. CF‑ified files are under:", OUTPUT_ROOT)

Processing year 1981 → /scratch/tmiesse/project/data4spatial/1981/fort.63.cf.nc
  → Done 1981
All years processed. CF‑ified files are under: /scratch/tmiesse/project/data4spatial
